In [1]:
import sys
sys.path.append("../")

import matplotlib.pyplot as plt
import numpy as np
from src.loaders import load_data, load_labels
from src.classify import MCM_Classifier
from src.plot import  plot_results
import os
from src.plot import plot_confusion_matrix, plot_label_prob_diff

In [2]:
# Customizable environment variables
n_categories = 10  # Number of categories to be classified
n_variables = 121  # Number of variables in the dataset
mcm_filename_format = "train-images-unlabeled-{}_comms.dat"
data_filename_format = "train-images-unlabeled-{}.dat"
data_path = "../INPUT/data/"
communities_path = "../INPUT/MCMs/"
output_path = "../OUTPUT/"

In [ ]:
def arr_to_int(binary_arr):
    return binary_arr.dot(1 << np.arange(binary_arr.shape[-1] - 1, -1, -1))

def str

new method to extract how informative a community is at differentiating between two classes

In [19]:
comms3 = np.loadtxt(os.path.join(communities_path, "train-images-unlabeled-3_comms.dat"),dtype=str)
comms5 = np.loadtxt(os.path.join(communities_path, "train-images-unlabeled-5_comms.dat"),dtype=str)

In [26]:
5

In [25]:
# what is the probability of icc1_c3 for some random image
icc1_c3 = comms3[0]

def __construct_P(estimator="add_smooth",alp=1) -> tuple:
    """
    Construct probability distributions for each category.
    This function should only be run once during initalization of the classifier.

    Code provided by https://github.com/ebokai

    Args:
        P (list): List of the probability distributions for each category
        MCM (list): List of the MCMs for each category
        n_categories (int): Number of categories in the dataset
        n_variables (int): Number of variables in the dataset
    """
    MCM = []
    P = []

    print_box("Constructing probability distributions...")

    # if not self.__validate_input_comms():
        # raise ValueError("Input data folder file count does not match number of categories. Did you run the fit method?.")

    # Construct probability distributions for each category
    for k in range(self.n_categories):
        # Add MCM to list
        try:
            # mcm = load_mcm(f"../INPUT/MCMs/{self.__mcm_filename_format.format(k)}")
            mcm = load_mcm(os.path.join(self.comms_path, self.__mcm_filename_format.format(k)))
            MCM.append(mcm)
        except:
            # Throw error if MCM file not found
            raise FileNotFoundError(f"Could not find MCM file for category {k}")

        # Load data
        try:
            # data = load_data(f"../INPUT/data/{self.__data_filename_format.format(k)}")
            data = load_data(os.path.join(self.data_path, self.__data_filename_format.format(k)))
        except:
            # Throw error if data file not found
            raise FileNotFoundError(f"Could not find data file for category {k}")
        pk = []

        for icc in mcm:
            # TODO maybe we later would like a single call to our estimator function
            #  here if we are also using some KL-divergence-based approach that works very differently
            idx = [i for i in range(self.n_variables) if icc[i] == "1"]
            rank = len(idx)

            # p_icc = np.zeros(2**rank)
            # p_icc = np.full(2**rank,fill_value=np.nan) # initialized with 0, lets try nan
            # p_icc = np.full(2**rank,fill_value=1/(2**rank+2)) # laplacian smoothing for 0 observations: (0+1)/(N+2*1)
            p_icc = self.estimator_init(rank,method=estimator,alpha=alp) # TODO this takes a lot of space, maybe we could find a sparse replacement

            icc_data = data[:, idx]
            icc_strings = [
                int("".join([str(s) for s in state]), 2) for state in icc_data
            ]

            u, c = np.unique(icc_strings, return_counts=True)
            p_icc[u] = self.estimator_prob(c,method=estimator,alpha=alp)
            # p_icc[u] = c / np.sum(c)

            pk.append(p_icc)

        P.append(pk)

    self.__P = P
    self.__MCM = MCM

    return self.__P, self.__MCM


121

In [ ]:

# what is the average probability distribution for icc1_c3. That is i give it all 3s and it gives me a pdf.
# what is the same over all 5s?

# what is the KL diveregence between both? is this a measure of how informative this icc is?
# what could be a measure of how destinct an ICC is for a class?